In [1]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split

# ---------- giả lập data ----------
# Thay X_raw, y_raw bằng dữ liệu thật của bạn
np.random.seed(0)
n_samples, n_features = 1480, 81
X_raw = np.random.randn(n_samples, n_features)
y_raw = X_raw[:, :3].sum(axis=1) + 0.5 * (X_raw[:, 3]**2) + np.random.randn(n_samples) * 0.5
y_raw = y_raw.reshape(-1, 1)

# ---------- tiền xử lý ----------
degree = 2
poly = PolynomialFeatures(degree=degree, include_bias=False)  # không thêm bias ở đây
X_poly = poly.fit_transform(X_raw)  # có thể lớn: kiểm tra X_poly.shape
scaler = StandardScaler()
X = scaler.fit_transform(X_poly)  # chuẩn hóa features
y = (y_raw - y_raw.mean()) / y_raw.std()  # (tuỳ chọn) chuẩn hóa target

m, d = X.shape
print("After poly transform:", X.shape)

# ---------- split ----------
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# ---------- thêm bias column (intercept) ----------
def add_bias(X):
    return np.c_[np.ones((X.shape[0], 1)), X]

X_train_b = add_bias(X_train)
X_val_b = add_bias(X_val)

# ---------- hyperparams ----------
epochs = 100
batch_size = 64
eta0 = 0.01         # initial learning rate
alpha = 0.001       # overall regularization strength
l1_ratio = 0.5      # ElasticNet mixing: 0->L2 only, 1->L1 only

# ---------- init params ----------
rng = np.random.RandomState(1)
theta = rng.randn(X_train_b.shape[1], 1) * 0.01  # column vector (d+1, 1)

# ---------- helper: compute loss (MSE + ElasticNet) ----------
def compute_loss(Xb, y, theta, alpha, l1_ratio):
    m = Xb.shape[0]
    preds = Xb @ theta
    mse = 0.5 * np.mean((preds - y)**2)
    # Không regularize bias (theta[0])
    theta_no_bias = theta.copy()
    theta_no_bias[0] = 0.0
    l2 = 0.5 * (1 - l1_ratio) * np.sum(theta_no_bias**2) * alpha
    l1 = alpha * l1_ratio * np.sum(np.abs(theta_no_bias))
    return mse + l1 + l2

# ---------- training loop: mini-batch GD with ElasticNet (subgradient for L1) ----------
history = {"train_loss": [], "val_loss": []}
n_train = X_train_b.shape[0]
n_batches = int(np.ceil(n_train / batch_size))

for epoch in range(1, epochs + 1):
    # shuffle
    perm = np.random.permutation(n_train)
    X_shuf = X_train_b[perm]
    y_shuf = y_train[perm]

    # learning rate schedule (simple)
    eta = eta0 / np.sqrt(epoch)  # decay

    for b in range(n_batches):
        start = b * batch_size
        end = min(start + batch_size, n_train)
        Xb = X_shuf[start:end]
        yb = y_shuf[start:end]

        m_batch = Xb.shape[0]
        preds = Xb @ theta  # (m_batch,1)
        grad_mse = (Xb.T @ (preds - yb)) / m_batch  # (d+1,1)

        # regularization gradient (do not regularize bias term)
        theta_no_bias = theta.copy()
        theta_no_bias[0] = 0.0

        # L2 part gradient: alpha * (1 - l1_ratio) * theta_no_bias
        grad_l2 = alpha * (1 - l1_ratio) * theta_no_bias

        # L1 part subgradient: alpha * l1_ratio * sign(theta_no_bias)
        # subgradient at 0 can be any value in [-1,1]; we use 0 for stability
        sign = np.sign(theta_no_bias)
        sign[theta_no_bias == 0.0] = 0.0
        grad_l1 = alpha * l1_ratio * sign

        grad = grad_mse + grad_l2 + grad_l1

        # update
        theta -= eta * grad

    # compute losses
    train_loss = compute_loss(X_train_b, y_train, theta, alpha, l1_ratio)
    val_loss = compute_loss(X_val_b, y_val, theta, alpha, l1_ratio)
    history["train_loss"].append(train_loss)
    history["val_loss"].append(val_loss)

    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:3d}  train_loss={train_loss:.4f}  val_loss={val_loss:.4f}")

# ---------- kết quả ----------
print("Done. Final train/val loss:", history["train_loss"][-1], history["val_loss"][-1])


After poly transform: (1480, 3402)
Epoch   1  train_loss=0.1449  val_loss=0.5189
Epoch  10  train_loss=0.0268  val_loss=0.4680
Epoch  20  train_loss=0.0199  val_loss=0.4608
Epoch  30  train_loss=0.0177  val_loss=0.4564
Epoch  40  train_loss=0.0167  val_loss=0.4528
Epoch  50  train_loss=0.0161  val_loss=0.4497
Epoch  60  train_loss=0.0157  val_loss=0.4467
Epoch  70  train_loss=0.0154  val_loss=0.4440
Epoch  80  train_loss=0.0152  val_loss=0.4414
Epoch  90  train_loss=0.0151  val_loss=0.4389
Epoch 100  train_loss=0.0149  val_loss=0.4366
Done. Final train/val loss: 0.014911878079776704 0.43657872587092944


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer  # hoặc IterativeImputer
import shap

# =========================================================
# 1. GIẢ LẬP DỮ LIỆU (thay phần này bằng dữ liệu thật)
# =========================================================
np.random.seed(42)
X = np.random.rand(1480, 10)
y = 5 * X[:, 0]**2 + 3 * X[:, 1] + np.random.randn(1480) * 0.3

# Tạo một vài giá trị NaN để test imputer
mask = np.random.rand(*X.shape) < 0.05
X[mask] = np.nan

# =========================================================
# 2. XÂY DỰNG PIPELINE
# =========================================================
pipeline = Pipeline([
    ("imputer", KNNImputer(n_neighbors=5)),
    ("poly", PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)),
    ("scaler", StandardScaler()),
    ("regressor", SGDRegressor(
        penalty="elasticnet",
        max_iter=2000,
        tol=1e-3,
        alpha=0.0005,
        l1_ratio=0.15,
        learning_rate="invscaling",
        eta0=0.01,
        random_state=42
    ))
])

# =========================================================
# 3. SCORING FUNCTIONS
# =========================================================
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

scoring = {
    "RMSE": make_scorer(rmse, greater_is_better=False),
    "MAE": make_scorer(mean_absolute_error, greater_is_better=False),
    "R2": make_scorer(r2_score)
}

# =========================================================
# 4. GRID SEARCH + CROSS VALIDATION
# =========================================================
param_grid = {
    "poly__degree": [1, 2, 3],
    "poly__interaction_only": [False, True],
    "regressor__alpha": [0.0001, 0.001, 0.01],
    "regressor__l1_ratio": [0.1, 0.5, 0.9]
}

grid = GridSearchCV(
    pipeline,
    param_grid,
    scoring=scoring,
    refit="R2",
    cv=5,
    n_jobs=-1,
    verbose=1
)

grid.fit(X, y)

# =========================================================
# 5. ĐÁNH GIÁ KẾT QUẢ
# =========================================================
print("==== BEST PARAMETERS ====")
print(grid.best_params_)
print()

best_model = grid.best_estimator_

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

best_model.fit(X_train, y_train)
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

train_rmse = rmse(y_train, y_pred_train)
test_rmse = rmse(y_test, y_pred_test)

print("==== FINAL EVALUATION ====")
print(f"Train RMSE : {train_rmse:.4f}")
print(f"Test  RMSE : {test_rmse:.4f}")
print(f"Train R²   : {r2_score(y_train, y_pred_train):.4f}")
print(f"Test  R²   : {r2_score(y_test, y_pred_test):.4f}")

# =========================================================
# 6. GIẢI THÍCH BẰNG SHAP
# =========================================================
# (Chỉ thực hiện nếu bạn có shap đã cài)
try:
    explainer = shap.Explainer(best_model.named_steps["regressor"], best_model[:-1].transform(X_train))
    shap_values = explainer(best_model[:-1].transform(X_test[:100]))
    shap.summary_plot(shap_values, best_model[:-1].transform(X_test[:100]), show=False)
except Exception as e:
    print("SHAP explanation skipped:", e)
